In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


2025-08-18 18:31:42.180041: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755534702.199198   51318 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755534702.204406   51318 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755534702.217054   51318 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755534702.217073   51318 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755534702.217083   51318 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Importy

In [2]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model


## Wczytanie modelu

In [3]:
model = load_model('../model/2025-08-18/lstm.keras')

I0000 00:00:1755534710.954422   51318 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1733 MB memory:  -> device: 0, name: NVIDIA GeForce MX150, pci bus id: 0000:01:00.0, compute capability: 6.1


## Wczytanie etykiet z pliku tekstowego


In [4]:
with open('../model/2025-07-14//labels.txt', 'r') as f:
    label_names = [line.strip() for line in f.readlines()]

## MediaPipe inicjalizacja

In [5]:
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False, model_complexity=1)
mp_drawing = mp.solutions.drawing_utils

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Bufor sekwencji

In [6]:
sequence = []
SEQ_LENGTH = 30
THRESHOLD = 0.8

## Normalizacja względem nadgarstka pierwszej klatki

In [7]:
def normalize_sequence(sequence_array):
    sequence_array = np.array(sequence_array)
    origin = sequence_array[0][0]  # Nadgarstek 1. klatki
    return sequence_array - origin

## Ekstrakcja 21 punktów prawej dłoni

In [8]:
def extract_hand_landmarks(results):
    if results.right_hand_landmarks:
        return np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark])
    return None

# Kamera na żywo

In [10]:
# Kamera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(image_rgb)

    # Rysowanie dłoni
    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    hand_landmarks = extract_hand_landmarks(results)

    if hand_landmarks is not None:
        sequence.append(hand_landmarks)

        if len(sequence) == SEQ_LENGTH:
            norm_seq = normalize_sequence(sequence)
            input_seq = norm_seq.reshape(1, SEQ_LENGTH, 63)

            pred = model.predict(input_seq)[0]
            max_prob = np.max(pred)
            label = label_names[np.argmax(pred)]

            if max_prob > THRESHOLD:
                cv2.putText(frame, f'{label} ({max_prob:.2f})', (10, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            sequence.pop(0)

    else:
        sequence = []

    cv2.imshow('Real-Time Sign Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


W0000 00:00:1755534071.869491   23823 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
I0000 00:00:1755534075.368009   23735 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━

# Kamera na żywo z cooldown'em

## Parametry

In [9]:
sequence = []
recognized_text = ""
last_prediction_time = 0
cooldown_seconds = 3
SEQ_LENGTH = 30
THRESHOLD = 0.8

In [10]:
import time

# Kamera
cap = cv2.VideoCapture(0)
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    if frame_count % 2 != 0:
        continue
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(image_rgb)
    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    hand_landmarks = extract_hand_landmarks(results)

    current_time = time.time()
    time_since_last = current_time - last_prediction_time
    cooldown_remaining = max(0, cooldown_seconds - time_since_last)

    if hand_landmarks is not None:
        sequence.append(hand_landmarks)

        if len(sequence) == SEQ_LENGTH:
            norm_seq = normalize_sequence(sequence)
            input_seq = norm_seq.reshape(1, SEQ_LENGTH, 63)

            pred = model.predict(input_seq, verbose=0)[0]
            max_prob = np.max(pred)
            label = label_names[np.argmax(pred)]

            if max_prob > THRESHOLD and cooldown_remaining == 0:
                recognized_text += label
                last_prediction_time = current_time

            sequence.pop(0)
    else:
        sequence = []

    # Pasek cooldownu
    bar_x, bar_y = 10, 80
    bar_width, bar_height = 300, 20
    progress = int((1 - cooldown_remaining / cooldown_seconds) * bar_width)

    bar_color = (0, 255, 0) if cooldown_remaining == 0 else (0, 0, 255)
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (255, 255, 255), 2)
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + progress, bar_y + bar_height), bar_color, -1)
    cv2.putText(frame, f'Cooldown: {cooldown_remaining:.1f}s', (bar_x, bar_y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, bar_color, 2)

    # Napis
    cv2.putText(frame, f'Text: {recognized_text}', (10, 130),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    cv2.imshow('Real-Time Sign Recognition (Cooldown)', frame)
    if cv2.waitKey(1) & 0xFF == ord('c'):
        sequence = []
        recognized_text = ""
    if cv2.waitKey(1) & 0xFF == ord('q'):
        sequence = []
        recognized_text = ""
        break

cap.release()
cv2.destroyAllWindows()


W0000 00:00:1755534741.440822   51425 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
I0000 00:00:1755534747.218273   51382 cuda_dnn.cc:529] Loaded cuDNN version 90300


KeyboardInterrupt: 